# Homework 3

### Due Date: Tuesday, October 30, 2018

### Getting Started

- Training optimal policies / value functions can take very long, so be sure to start solving the problems early to give yourself enough time to finish training everything.
- Please save any plots you generate and add them to the notebook. We will not rerun your code for you.

# Question 1: Direct Policy Algorithms

While in previous assignments we focused primarily on value iteration algorithms (e.g. DQN), in this homework we will be implementing direct policy algorithms, which simply learn the parameters of your mapping from states to actions. The structure of these algorithms are essentially the same as those we implemented previously, with the core constituents being a function approximator, a method for computing actions, a method for collecting samples, and a training operation. Accordingly, in order to recycle aspects of the code, we choose to organize these methods into a parent class which may then be abstracted and recycled for the algorithms we implement in subsequent problems.

In the below cell, we have instantiated this class and filled in the components you have already written up in homework 2. Spend some time reviewing this class, and then complete the following missing operations:

1. Modify the `create_model` method to output a trainable variable (i.e. `tf.get_variable`) that computes the logstd of a certain action in the case of stochastic policies. For this assignment, we will assume that this variable is not dependent on the input state. We will use this in problems 2 and 3.
2. Modify in the `action_op` method to calculate and return the actions based on the current parametrization of the policy in the stochastic setting (mean, logstd). The action should be: $\pi(s) = \pi_\text{mean}(s) + exp(logstd) * \mathcal{N}(0,1)$.
3. Test your implemenations in the cells immediately below this class! This should help you do this incrementally.

**Hints**: 

- Refer to the `__init__` method when choosing the placeholders and dimensions for your augmentations to the above methods. 
- Some useful tensorflow classes and methods include: `tf.get_variable`, `tf.exp`, `tf.random_normal`.

In [ ]:
import tensorflow as tf
import tensorflow.contrib.layers as layers
import numpy as np
import os


class DirectPolicyAlgorithm(object):

    def __init__(self, 
                 env,
                 linear=False,
                 stochastic=False,
                 hidden_size=[64, 64], 
                 nonlinearity=tf.nn.relu):
        """Instantiate the policy iteration class.

        This initializes the policy model with a set of trainable 
        parameters, and creates a tensorflow session and a saver 
        to save checkpoints during training.

        In order to train an algorithm using this class, type:

            >>> alg = DirectPolicyAlgorithm(...)
            >>> alg.train(...)

        Note that the "train" method is abstract, and needs to be
        filled in by a child class.

        Attributes
        ----------
        env : gym.Env
            the environment that the policy will be trained on
        linear : bool, optional
            specifies whether to use a linear or neural network 
            policy, defaults to False (i.e. use a fully-connected
            network)
        stochastic : bool, optional
            specifies whether to use a stochastic or deterministic 
            policy, defaults to False (i.e. deterministic policy)
        hidden_size : list of int, optional
            list of hidden layers, with each value corresponding 
            to the number of nodes in that layer 
        nonlinearity : tf.nn.*
            activation nonlinearity
        """
        # clear any previous computation graph
        tf.reset_default_graph()
        
        # set a random seed
        tf.set_random_seed(1234)
        
        # start a tensorflow session
        self.sess = tf.Session()
        
        # environment to train on
        self.env = env

        # number of elements in the action space
        self.ac_dim = env.action_space.shape[0]
        # number of elements in the observation space
        self.obs_dim = env.observation_space.shape[0]

        # mean state (substracted from the state before computing 
        # actions). This is used in problem 3.
        self.mean_state = np.zeros(self.obs_dim)

        # actions placeholder
        self.a_t_ph = tf.placeholder(dtype=tf.float32, 
                                     shape=[None, self.ac_dim])
        # state placeholder
        self.s_t_ph = tf.placeholder(dtype=tf.float32, 
                                     shape=[None, self.obs_dim])
        # expected reward placeholder
        self.rew_ph = tf.placeholder(dtype=tf.float32, 
                                     shape=[None])

        # specifies whether the policy is stochastic
        self.stochastic = stochastic

        # policy that the agent executes during training/testing
        self.policy = self.create_model(
            args={
                "num_actions": self.ac_dim,
                "hidden_size": hidden_size,
                "linear": linear,
                "nonlinearity": nonlinearity,
                "stochastic": stochastic,
                "scope": "policy",
            }
        )

        self.symbolic_action = self.action_op()

        # initialize all variables
        self.sess.run(tf.global_variables_initializer())

        # create saver to save model variables
        self.saver = tf.train.Saver()

    def create_model(self, args):
        """Create a model for your policy or other components.

        This model may be linear or a fully-connected network. In
        addition, a logstd variable may be specified if the policy
        is stochastic, otherwise, the logstd output is set to None.

        Parameters
        ----------
        args : dict
            model-specific arguments, with keys:
              - "stochastic": a boolean operator that specifies
                whether the policy is meant to be stoachstic or
                deterministic. If it is stochastic, an additional
                trainable variable is created to compute the logstd 
                of an action given. This variable is not dependent 
                on the input state.
              - "hidden_size": a list that specified the shape 
                of the neural networ (if "linear" is False)
              - "num_actions" number of output actions
              - "scope": scope of the model

        Returns
        -------
        tf.Variable
            mean actions of the policy
        tf.Variable or None
            logstd of the policy actions. If the policy is deterministic,
            this term is None
        """
        with tf.variable_scope(args["scope"]):
            # create the hidden layers
            last_layer = self.s_t_ph
            for i, hidden_size in enumerate(args["hidden_size"]):
                last_layer = tf.layers.dense(
                    inputs=last_layer,
                    units=hidden_size,
                    activation=args["nonlinearity"])

            # create the output layer
            output_mean = tf.layers.dense(
                inputs=last_layer,
                units=args["num_actions"],
                activation=None)

        if args["stochastic"]:
            ################################################################
            # Create a trainable variable whose output is the same size    #
            # as the action space. This variable will represent the output #
            # log standard deviation of your stochastic policy.            #
            #                                                              #
            # Refer to the __init__ method in when choosing an appropriate #
            # shape of your variable.                                      #
            #                                                              #
            # Note: To create this variable, use the `tf.get_variable`     #
            # function.                                                    #
            ################################################################
            output_logstd =  ### FILL IN ###
        else:
            output_logstd = None

        return output_mean, output_logstd
    
    def action_op(self):
        """Create a symbolic expression that will be used to compute 
        actions from observations.

        If the policy is determistic, the action is simply dictated by
        the output of the policy mean.

        Alternatively, if the policy is stochastic, the action is:

            a_t = output_mean + exp(output_logstd) * z; z ~ N(0,1)
        """
        if self.stochastic:
            output_mean, output_logstd = self.policy

            ##############################################################
            # Implement a stochastic version of computing actions.       #
            #                                                            #
            # Remember, the action in a stochastic policy represented by #
            # a diagonal Gaussian distribution with mean "M" and log     #
            # standard deviation "lstd" is computed as follows:          #
            #                                                            #
            #     a = M + exp(lstd) * z                                  #
            #                                                            #
            # where z is a random normal value, i.e. z ~ N(0,1)          #
            #                                                            #
            # In order to generate numbers from a normal distribution,   #
            # use the `tf.random_normal` function.                       #
            ##############################################################
            symbolic_action = ### FILL IN ###
        else:
            symbolic_action, _ = self.policy
        
        return symbolic_action

    def compute_action(self, obs):
        """Returns a list of actions for a given observation.

        Parameters
        ----------
        obs : np.ndarray
            observations

        Returns
        -------
        np.ndarray
            actions by the policy for a given observation
        """
        return self.sess.run(self.symbolic_action, 
                             feed_dict={self.s_t_ph: obs})

    def rollout(self):
        """Collect samples from one rollout of the policy.

        Returns
        -------
        dict
            dictionary conta.ining trajectory information for the rollout,
            specifically containing keys for "state", "action", 
            "next_state", "reward", and "done"
        """
        states = []
        next_states = []
        actions = []
        rewards = []
        dones = []

        # start a new rollout by resetting the environment and 
        # collecting the initial state
        state =  self.env.reset()

        steps = 0
        while True:
            steps += 1

            # compute the action given the state
            action = self.compute_action([state])
            action = action[0]

            # advance the environment once and collect the next state, 
            # reward, done, and info parameters from the environment
            next_state, reward, done, info =  self.env.step(action)

            # add to the samples list
            states.append(state)
            actions.append(action)
            next_states.append(next_state)
            rewards.append(reward)
            dones.append(done)

            state = next_state

            # if the environment returns a True for the done parameter,
            # end the rollout before the time horizon is met
            if done or steps > env._max_episode_steps:
                break

        # create the output trajectory
        trajectory = {"state": np.array(states, dtype=np.float32),
                      "reward": np.array(rewards, dtype=np.float32),
                      "action": np.array(actions, dtype=np.float32),
                      "next_state": np.array(next_states, dtype=np.float32),
                      "done": np.array(dones, dtype=np.float32)}

        return trajectory

    def train(self, args):
        """Abstract training method.

        This method will be filled in by algorithm-specific
        training operations in subsequent problems.

        Parameters
        ----------
        args : dict
            algorithm-specific hyperparameters
        """
        raise NotImplementedError

    def save_checkpoint(self, filename):
        """Save a checkpoint for later viewing."""
        current_dir = os.getcwd()
        save_loc = os.path.join(current_dir, filename)
        self.saver.save(self.sess, save_loc)

    def load_checkpoint(self, filename):
        """Load the model from a specific checkpoint."""
        current_dir = os.getcwd()
        save_loc = os.path.join(current_dir, filename)
        self.saver.restore(self.sess, save_loc)

In order to test your implementation of the **stochastic policy**, run the below cell.

Note that if you are receiving the warning:
    
    WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.

don't worrry, this is meant to happen.

In [ ]:
from hw3_tests import test_1_stochastic
test_1_stochastic(DirectPolicyAlgorithm)

In order to test your implementation of **sampling actions** from the stochastic policy, run the below cell.

In [ ]:
from hw3_tests import test_1_stochastic_sampling
test_1_stochastic_sampling(DirectPolicyAlgorithm)

# Question 2: Policy Gradient Methods

In this section, we will implement the likelihood ratio (or REINFORCE) algorithm presented in the lectures. As a review, the objective here (and in all direct policy algorithms) is optimize the parameters $\theta$ of some policy $\pi_\theta$ so that the expected return

\begin{equation}
J(\theta) = \mathbb{E} \bigg\{ \sum_{t=0}^T \gamma^t r(t) \bigg\}
\end{equation}

is optimized. In this algorithm, this is done by calculating the gradient $\nabla_\theta J$ and applying a gradient descent method to find a better policy.

\begin{equation}
\theta ' = \theta + \alpha \nabla_\theta J(\theta)
\end{equation}

In order compute this gradient, we generate trajectories $\tau$ from a system by rollouts within an an environment characterized a model transition probability $p$, i.e. $\tau \sim p_\theta(\tau) = p(\tau|\theta)$ with discounted return $r(\tau) = \sum_{i=0}^T \gamma^ir_i$. The expected return is then:

\begin{equation}
J(\theta) = \mathbb{E} \{ r(\tau) \} = \int_{\mathbb{T}} p_\theta(\tau)r(\tau) d\tau
\end{equation}

Now, in order to compute the gradient, we apply the log trick and get an expression as follows:

\begin{equation}
\nabla_\theta J(\theta) = \int_{\mathbb{T}} \nabla_\theta p_\theta(\tau)r(\tau) d\tau = \int_{\mathbb{T}} p_\theta(\tau) \nabla_\theta \log p_\theta(\tau)r(\tau) d\tau = \mathbb{E} \{ \nabla_\theta \log p_\theta(\tau) r(\tau) \}
\end{equation}

This expression seems to still contain the model; however, knowing that the probability of a given trajectory can be expressed as:

\begin{equation}
p_\theta(\tau) = p(s_0) \prod_{t=0}^Tp(s_{t+1}|s_t,a_t)\pi_\theta(a_t|s_t)
\end{equation}

The corresponding gradient of this log transition is simply:

\begin{equation}
\nabla_\theta \log p_\theta (\tau) = \sum_{t=0}^T \nabla_\theta \log \pi_\theta (a_t | s_t)
\end{equation}

This means we can rewrite our policy gradient as:

\begin{equation}
\nabla_\theta J (\theta) = \frac{1}{N} \sum_{i=0}^{N} \bigg( \sum_{t=0}^{T} \nabla_\theta \log \pi_\theta (a_{it} | s_{it}) \bigg) \bigg( \sum_{t=0}^T \gamma^{t}r_i(t) \bigg)
\end{equation}

and the model disappears! Finally, taking into account the causality principle discussed in class, a less variance-prone expression for this gradient is:

\begin{equation}
\nabla_\theta J (\theta) = \frac{1}{N} \sum_{i=0}^{N} \sum_{t=0}^{T} \nabla_\theta \log \pi_\theta (a_{it} | s_{it}) \sum_{t'=t}^T \gamma^{t'-t}r_i(t')
\end{equation}

This final expression should be the major take-away from this review, and what you will be implementing in the subsequent section.

### a. REINFORCE

In this section, we will implement the basic form of the REINFORCE algorithm, which goes as follows:

1. Collect samples from your current representation of the policy $\pi_\theta(s)$ by executing a few rollouts of the environment.
2. Compute an estimate for the expected return from any state $s_t$. In order to do so, we will the causality principle, also called *reward-to-go*, which represents the estimated return as the sum of discounted returns starting from the state and moving forward.
3. Compute the log-likelihood of each action that was performed by the policy at every given step.
4. Estimate and apply a gradient to the training parameters for the policy using the representation of the gradient presented at the start of this problem. 
5. Repeat steps 1-4 for a certain number of training iterations.

The below cell provides the skeleton key for the above algorithm, with four new methods specified: `log_likelihood`, `compute_expected_return`, `update_parameters`, and `apply_gradients`. Fill in the missing elements of this algorithm.

**Hint**: Some useful tensorflow classes and methods include: `tf.contrib.distributions.MultivariateNormalDiag`, `tf.reduce_mean`, `tf.multiply`.

In [ ]:
import tensorflow as tf
import numpy as np
import time

class REINFORCE(DirectPolicyAlgorithm):

    def train(self,
              num_iterations=100,
              steps_per_iteration=1000,
              learning_rate=0.001,
              gamma=0.95, 
              **kwargs):
        """Perform the REINFORE training operation.

        Parameters
        ----------
        num_iterations : int
            number of training iterations
        steps_per_iteration : int
            number of individual samples collected every training
            iteration
        learning_rate : float
            optimizer learning rate
        gamma : float
            discount rate
        kwargs : dict
            additional arguments

        Returns
        -------
        list of float
            average return per iteration
        """
        # set the discount as an attribute
        self.gamma = gamma
        
        # set the learning rate as an attribute
        self.learning_rate = learning_rate

        # create a symbolic expression to compute the log-likelihoods 
        log_likelihoods = self.log_likelihoods()

        # create a symbolic expression for updating the parameters of 
        # your policy
        #
        # Note: the second operator will be used in problem 2.b, please 
        # ignore when solving 2.a
        self.opt, self.opt_baseline = self.define_updates(log_likelihoods)

        # initialize all variables
        self.sess.run(tf.global_variables_initializer())

        # average return per training iteration
        ret_per_iteration = []
        
        samples = []
        for i in range(num_iterations):
            # collect samples from the current policy
            samples.clear()
            steps_so_far = 0
            while steps_so_far < steps_per_iteration:
                new_samples = self.rollout()
                steps_so_far += new_samples["action"].shape[0]
                samples.append(new_samples)

            # compute the expected returns
            v_s = self.compute_expected_return(samples)

            # compute and apply the gradients
            self.call_updates(log_likelihoods, samples, v_s, **kwargs)

            # compute the average cumulative return per iteration
            average_rew = np.mean([sum(s["reward"]) for s in samples])

            # display iteration statistics
            print("Iteration {} return: {}".format(i, average_rew))
            ret_per_iteration.append(average_rew)

        return ret_per_iteration

    def log_likelihoods(self):
        """Create a tensorflow operation that computes the log-likelihood 
        of each performed action.

        Remember, the actions in this case are not deterministic, but 
        rather sampled from a Gaussian distribution; accordingly, there 
        is a probability associated with each action occuring.
        """
        # tensors representing the mean and standard deviation of
        # performing a specific action given a state (these are the 
        # parameters of your policy)
        output_mean, output_logstd = self.policy

        ##############################################################
        # Create a tf operation to compute the log-likelihood of     #
        # each action that was performed by the policy during the    #
        # trajectories.                                              #
        #                                                            #
        # The log likelihood in the continuous case where the policy #
        # is expressed by a multivariate gaussian can be computing   #
        # using the tensorflow object:                               #
        #                                                            #
        #    p = tf.contrib.distributions.MultivariateNormalDiag(    #
        #        loc=...,                                            #
        #        scale_diag=...,                                     #
        #    )                                                       #
        #                                                            #
        # This method takes as input a mean (loc) and standard       #
        # deviation (scale_diag), and then can be used to compute    #
        # the log-likelihood of a variable as follows:               #
        #                                                            #
        #    log_likelihoods = p.log_prob(...)                       #
        #                                                            #
        # For this operation, you will want to use placeholders      #
        # created in the __init__ method of problem 1.               #
        ##############################################################
        log_likelihoods =  ### FILL IN ###

        return log_likelihoods

    def compute_expected_return(self, samples):
        """Compute the expected return from a given starting state.

        This is done by using the reward-to-go method.

        Parameters
        ----------
        rewards : list of list of float
            a list of N trajectories, with each trajectory contain T 
            returns values (one for each step in the trajectory)

        Returns
        -------
        list of float, or np.ndarray
            expected returns for each step in each trajectory
        """
        rewards = [s["reward"] for s in samples]

        ##############################################################
        # Estimate the expected return from any given starting state #
        # using the reward-to-go method.                             #
        #                                                            #
        # Using this method, the reward is estimated at every step   #
        # of the trajectory as follows:                              #
        #                                                            #
        #   r = sum_{t'=t}^T gamma^(t'-t) * r_{t'}                   #
        #                                                            #
        # where T is the time horizon at t is the index of the       #
        # current reward in the trajectory. For example, for a given #
        # set of rewards r = [1,1,1,1] and discount rate gamma = 1,  #
        # the expected reward-to-go would be:                        #
        #                                                            #
        #   v_s = [4, 3, 2, 1]                                       #
        #                                                            #
        # You will be able to test this in one of the cells below!   #
        ##############################################################
        v_s = ### FILL IN ###
        
        return v_s

    def define_updates(self, log_likelihoods):
        """Create a tensorflow operation to update the parameters of 
        your policy.

        Parameters
        ----------
        log_likelihoods : tf.Operation
            the symbolic expression you created to estimate the log 
            likelihood of a set of actions

        Returns
        -------
        tf.Operation
            a tensorflow operation for computing and applying the 
            gradients to the parameters of the policy
        None
            the second component is used in problem 2.b, please ignore 
            for this problem
        """
        ##############################################################
        # Specify a loss function that can be used to compute the    #
        # gradient of denoted at the start of problem 2.             #
        #                                                            #
        # Note: remember we are trying to **maximize** this value    #
        #                                                            #
        # For this operation, you will want to use placeholders      #
        # created in the __init__ method of problem 1, as well as    #
        # the operations provided as inputs to this problem.         #
        ##############################################################
        loss =  ### FILL IN ###
        opt = tf.train.AdamOptimizer(self.learning_rate).minimize(loss)

        return opt, None

    def call_updates(self,
                     log_likelihoods,
                     samples,
                     v_s, 
                     **kwargs):
        """Apply the gradient update methods in a tensorflow session.

        Parameters
        ----------
        log_likelihoods: tf.Operation
            the symbolic expression you created to estimate the log 
            likelihood of a set of actions
        samples : list of dict
            a list of N trajectories, with each trajectory containing 
            a dictionary of trajectory data (see self.rollout)
        v_s : list of float, or np.ndarray
            the estimated expected returns from your
            `comput_expected_return` function 
        kwargs : dict
            additional arguments (used in question 3)
        """
        # concatenate the states
        states = np.concatenate([s["state"] for s in samples])

        # concatenate the actions
        actions = np.concatenate([s["action"] for s in samples])

        ##############################################################
        # Fill in the feed_dict component below to properly execute  #
        # the optimization step. Refer to the variables formed in    #
        # this problem as well as the __init__ method in problem 1   # 
        # when doing so.                                             #
        ##############################################################
        self.sess.run(self.opt, feed_dict={})  ### FILL IN ###

In order to test your implementation of the `log_likelihood` method, run the below cell.

In [ ]:
from hw3_tests import test_2_log_likelihood
test_2_log_likelihood(REINFORCE)

In order to test your implementation of the `compute_expected_return` method, run the below cell.

In [ ]:
from hw3_tests import test_2_expected_return
test_2_expected_return(REINFORCE)

### b. State-dependent Baselines

The estimates of the gradients for the REINFORCE algorithm presented above have high variance. One possible method for reducing this variance is to subtract a baseline from the advantage function. The new gradient is then:

\begin{equation}
\nabla_\theta J (\theta) = \frac{1}{N} \sum_{i=0}^{N} \sum_{t=0}^{T} \nabla_\theta \log \pi_\theta (a_{it} | s_{it}) \bigg( \sum_{t'=t}^T \gamma^{t'-t}r_i(t') - b(s_{it}) \bigg)
\end{equation}

As we proved in class, subtracting any state-dependent baselines $b(s_t)$ leads to an unbiased estimator of the gradient. Accordingly, we choose to use a neural network baseline $b_\phi(s_t)$ that estimates the expected return from any state $s_t$. This network is trained from trajectory data collected during the sample collection procedure to minimize the error between the model estimate of the expected return and the expected return acquired during training. It's loss is accordingly:

\begin{equation}
L_\phi = || b_\phi(s_t) - \sum_{t'=t}^T \gamma^{t'-t}r(t') ||^2
\end{equation}

The below cell provides a skeleton key for modifying the REINFORCE class to include a state-dependent baseline represented by a neural network (called `self.baseline`). Modify the `define_updates` and `call_updates` methods in this class.

**Hint**: Some useful tensorflow classes and methods include: `tf.reduce_mean`, `tf.multiply`, `tf.square`.

In [ ]:
class REINFORCE_baseline(REINFORCE):

    def train(self,
              num_iterations=100,
              steps_per_iteration=1000,
              learning_rate=0.001,
              gamma=0.95,
              **kwargs):
        """Perform the REINFORE with baselines training operation.

        Parameters
        ----------
        num_iterations : int
            number of training iterations
        steps_per_iteration : int
            number of individual samples collected every training
            iteration
        learning_rate : float
            optimizer learning rate
        gamma : float
            discount rate
        kwargs : dict
            additional arguments

        Returns
        -------
        list of float
            average return per iteration
        """
        # function approximator used for estimated the expected return 
        # from a given state
        self.v_s, _ = self.create_model(
            args={
                "num_actions": 1,
                "hidden_size": [32, 32],
                "linear": False,
                "nonlinearity": tf.nn.relu,
                "stochastic": False,
                "scope": "critic",
            }
        )
        self.v_s = tf.squeeze(self.v_s)

        # the rest of the train method is as before (plus some 
        # modifications to some of the functions it calls, see below)
        return super().train(num_iterations,
                             steps_per_iteration, 
                             learning_rate,
                             gamma, 
                             **kwargs)

    def define_updates(self, log_likelihoods):
        """Create a tensorflow expression for optimizing the parameters 
        of your neural network policy, as well as the baseline network
        that is used to estimate the expected return from a given state.

        Parameters
        ----------
        log_likelihoods : tf.Operation
            the symbolic expression you created to estimate the log 
            likelihood of a set of actions

        Returns
        -------
        tf.Operation
            a tensorflow operation for computing and applying the 
            gradients to the parameters of the policy
        tf.Operation
            a tensorflow operation for computing and applying the 
            gradients to the parameters of the baseline
        """
        # this will recreate the policy optimization operation from
        # section 2.a
        opt, _ = super().define_updates(log_likelihoods)

        ##############################################################
        # Specify the loss function for the baseline.                #
        #                                                            #
        # The baseline is supposed to accurately approximate the     #
        # expect return values given state information.              #
        #                                                            #
        # Using self.rew_ph for the actual expected returns and      #
        # self.v_s for the estimated expected return values.         #
        ##############################################################
        loss_baseline =  ### FILL IN ###

        opt_baseline = tf.train.AdamOptimizer(self.learning_rate).minimize(
            loss_baseline)

        return opt, opt_baseline

    def call_updates(self,
                     log_likelihoods,
                     samples,
                     v_s,
                     **kwargs):
        """Apply the gradient update methods in a tensorflow session.

        This will now also include applying the gradients to the parameters 
        of the baseline.

        Parameters
        ----------
        log_likelihoods: tf.Operation
            the symbolic expression you created to estimate the log 
            likelihood of a set of actions
        samples : list of dict
            a list of N trajectories, with each trajectory containing 
            a dictionary of trajectory data (see self.rollout)
        v_s : list of float, or np.ndarray
            the estimated expected returns from your
            `comput_expected_return` function 
        kwargs : dict
            additional arguments (used in question 3)
        """
        # concatenate the states
        states = np.concatenate([s["state"] for s in samples])

        # concatenate the actions
        actions = np.concatenate([s["action"] for s in samples])

        ##############################################################
        # Compute the baseline values and reparameterize them so     #
        # that their mean and standard deviations match those of the #
        # estimated returns v_s.                                     #
        #                                                            #
        # Then create a new "advantage" term:                        #
        #                                                            #
        #     advantages = expected_return - baseline_val            #
        ##############################################################
        advantages =  ### FILL IN ###

        ##############################################################
        # Fill in the feed_dict component below to properly execute  #
        # the policy optimization step. This time, use your computed #
        # advantages instead of the expected returns.                #
        ##############################################################
        self.sess.run(self.opt, feed_dict={})  ### FILL IN ###

        ##############################################################
        # Fill in the feed_dict component of below to properly       #
        # execute the baselien optimization step. Refer to the       #
        # variables formed in this problem as well as the __init__   #
        # method in problem 1 when doing so.                         #
        ##############################################################
        self.sess.run(self.opt_baseline, feed_dict={})  ### FILL IN ###

### c. Testing your algorithm

When you are ready, test your policy gradient algorithms on the *Pendulum-v0* environment in the cell below. Note that this cell runs the algorithm twice: once in the absence of baselines and another time in the presence of them, and then plots the training curves for the two algorithms. Discuss the performance of the algorithms.

**Note**: Your best policy should get a max return of a bit over -200.

In [ ]:
# set this number as 1 for testing your algorithm, and 3 for plotting
NUM_RUNS = 3

# ===========================================================================
# Do not modify below here
# ===========================================================================

# we will test the algorithms on the Pendulum-v0 gym environment
import gym
env = gym.make("Pendulum-v0")

# train on the REINFORCE algorithm
import numpy as np
r = []
for i in range(NUM_RUNS):
    print("\n==== Training Run {} ====".format(i))
    alg = REINFORCE(env, stochastic=True)
    res = alg.train(learning_rate=0.01, gamma=0.95, steps_per_iteration=15000)
    r.append(np.array(res))
    alg = None

# save results
np.savetxt("InvertedPendulum_results_R.csv", np.array(r), delimiter=",")

In [ ]:
# set this number as 1 for testing your algorithm, and 3 for plotting
NUM_RUNS = 3

# ===========================================================================
# Do not modify below here
# ===========================================================================

# we will test the algorithms on the Pendulum-v0 gym environment
import gym
env = gym.make("Pendulum-v0")

# train on the REINFORCE with baselines algorithm
import numpy as np
r = []
for i in range(NUM_RUNS):
    print("\n==== Training Run {} ====".format(i))
    alg = REINFORCE_baseline(env, stochastic=True)
    res = alg.train(learning_rate=0.01, gamma=0.95, steps_per_iteration=15000)
    r.append(np.array(res))
    alg = None

# save results
np.savetxt("InvertedPendulum_results_Rb.csv", np.array(r), delimiter=",")

In [ ]:
# collect saved results
import numpy as np
r1 = np.genfromtxt("InvertedPendulum_results_R.csv", delimiter=",")
r2 = np.genfromtxt("InvertedPendulum_results_Rb.csv", delimiter=",")
all_results = [r1, r2]
labels = ["REINFORCE", "REINFORCE with baselines"]

# plot the results
%matplotlib inline
from hw3_utils import plot_results
fig = plot_results(all_results, labels)

# Question 3: Actor-Critic Algorithms

The actor-critic algorithm presented in class is a natural extension to the REINFORCE algorithm with state-dependent baselines that you implemented in section 2.b. Given that the baseline we used in that problem is exactly the value function (i.e. the estimate of expected future returns from a start state), the $b_\phi (s_t)$ can simply be substituted with a $V_\phi^\pi(s_t)$ term and the gradient is then:

\begin{equation}
\nabla_\theta J (\theta) = \frac{1}{N} \sum_{i=0}^{N} \sum_{t=0}^{T} \nabla_\theta \log \pi_\theta (a_{it} | s_{it}) \bigg( \sum_{t'=t}^T \gamma^{t'-t}r_i(t') - V_\phi^\pi(s_{it}) \bigg)
\end{equation}


Moreover, estimated the expected return from a state as the reward at that state plus the estimate from the value function at the next state, the policy gradient for the actor-critic method is defined as:

\begin{equation}
\nabla_\theta J (\theta) = \frac{1}{N} \sum_{i=0}^{N} \sum_{t=0}^{T} \nabla_\theta \log \pi_\theta (a_{it} | s_{it}) \bigg( r_i(t) + \gamma (1-d_{it}) V_\phi^\pi(s_{i,t+1}) - V_\phi^\pi(s_{it})\bigg)
\end{equation}


Moreover, to update this value function estimate, we will now use our previous estimate of the value function, and the loss associated with it is then:

\begin{equation}
L_\phi = || V_\phi^\pi(s_t) - \big(r(t) + \gamma (1-d) V_\phi^\pi(s_{t+1}) \big) ||^2
\end{equation}

In the below cell, we modify the REINFORCE_baseline algorithm to support the actor-critic method mentioned above. Fill in the missing components (note that some of these components simply need to be copied over from the previous question).

In [ ]:
class ActorCritic(REINFORCE_baseline):

    def train(self,
              num_iterations=100,
              steps_per_iteration=1000,
              learning_rate=0.001,
              gamma=0.95,
              **kwargs):
        """Perform the REINFORE with baselines training operation.

        Parameters
        ----------
        num_iterations : int
            number of training iterations
        steps_per_iteration : int
            number of individual samples collected every training
            iteration
        learning_rate : float
            optimizer learning rate
        gamma : float
            discount rate
        kwargs : dict
            additional arguments, containing:
            - "num_value_estimates": blank
            - "num_train_steps_per_estimate: blank

        Returns
        -------
        list of float
            average return per iteration
        """        
        return super().train(num_iterations=num_iterations,
                             steps_per_iteration=steps_per_iteration,
                             learning_rate=learning_rate,
                             gamma=gamma,
                             **kwargs)

    def compute_expected_return(self, samples):
        """Compute the expected return from a given starting state.

        This is done by using the reward-to-go method.

        Parameters
        ----------
        samples : list of dict
            trajectory information that was obtained while collecting 
            during this training iteration

        Returns
        -------
        list of float, or np.ndarray
            expected returns for each step in each trajectory
        """
        # rewards from the trajectories
        rewards = np.concatenate([s["reward"] for s in samples])
        # states from the trajectories
        states = np.concatenate([s["state"] for s in samples])
        # next states from the trajectories
        next_states = np.concatenate([s["next_state"] for s in samples])
        # done masks from the trajectories
        done = np.concatenate([s["done"] for s in samples])

        ##############################################################
        # Compute the expected returns.                              #
        #                                                            #
        # For this, we will assume that the expected return starting #
        # from the next state is the estimate of the return by our   #
        # baseline in section 2.b (i.e. self.v_s). Denoting this     #
        # function as V, and continuing to use the baseline          #
        # subtraction approach for reducing variance, the advantage  #
        # is expressed as:                                           #
        #                                                            #
        #     adv = r_t + gamma * (1 - done_t) * V(s_{t+1})          #
        #                                                            #
        # where t is the current time index.                         #
        #                                                            #
        # Note: Compute these values explicity (do not return        #
        # tensorflow objects).                                       #
        ##############################################################
        v_s =  ### FILL IN ###
        
        return v_s

    def call_updates(self,
                     log_likelihoods,
                     samples,
                     v_s,
                     **kwargs):
        """Apply the gradient update methods in a tensorflow session.

        This will now also include applying the gradients to the parameters 
        of the baseline.

        Parameters
        ----------
        log_likelihoods: tf.Operation
            the symbolic expression you created to estimate the log 
            likelihood of a set of actions
        states : list of list of np.ndarray
            a list of N trajectories, with each trajectory contain T 
            state values (one for each step in the trajectory)
        actions : list of list of np.ndarray
            a list of N trajectories, with each trajectory contain T 
            actions values (one for each step in the trajectory)
        v_s : list of float, or np.ndarray
            the estimated expected returns from your
            `compute_expected_return` function 
        kwargs : dict
            additional arguments (used in question 3)
        """
        # some variables for actor-critic (see the docstring in the 
        # train method)
        num_value_estimates = kwargs["num_value_estimates"]
        num_train_steps_per_estimate = kwargs["num_train_steps_per_estimate"]

        # concatenate the rewards
        rewards = np.concatenate([s["reward"] for s in samples])

        # concatenate the states
        states = np.concatenate([s["state"] for s in samples])

        # concatenate the next states
        next_states = np.concatenate([s["next_state"] for s in samples])

        # concatenate the actions
        actions = np.concatenate([s["action"] for s in samples])

        # concatenate the done masks
        dones = np.concatenate([s["done"] for s in samples])

        #################################################################
        # Call the policy update method as you had done in question 2.b #
        #################################################################
        advantages =  ### FILL IN ###

        self.sess.run(self.opt, feed_dict={})  ### FILL IN ###

        ##############################################################
        # Use the bootstrapped estimates of the expected return to   #
        # modify the parameters of your critic self.v_s.             #
        #                                                            #
        # In order to do so, first use your current parametrization  #
        # of the value function to estimate the expected future      #
        # return. Then, for a number of steps, update the parameters #
        # of your value function based on the loss described at the  #
        # start of this problem.                                     #
        ##############################################################
        for _ in range(num_value_estimates):
            target =  ### FILL IN ###
            for _ in range(num_train_steps_per_estimate):
                self.sess.run(self.opt_baseline,
                              feed_dict={self.rew_ph: target,
                                         self.s_t_ph: states})

When you are ready, test your actor-critic algorithm on the *Pendulum-v0* environment in the cell below.

**Note**: Your best policy should get a max return of a bit over -150.

In [ ]:
# set this number as 1 for testing your algorithm, and 3 for plotting
NUM_RUNS = 3

# ===========================================================================
# Do not modify below here
# ===========================================================================

# we will test the algorithms on the InvertedPendulum-v2 gym environment
import gym
env = gym.make("Pendulum-v0")

# train on the Actor Critic algorithm
import numpy as np
r = []
for i in range(NUM_RUNS):
    print("\n==== Training Run {} ====".format(i))
    alg = ActorCritic(env, stochastic=True)
    res = alg.train(learning_rate=0.01, gamma=0.95, steps_per_iteration=15000,
                    num_value_estimates=10, num_train_steps_per_estimate=10)
    r.append(np.array(res))
    alg = None

# save results
np.savetxt("InvertedPendulum_results_AC.csv", np.array(r), delimiter=",")

In [ ]:
# collect saved results
import numpy as np
r = np.genfromtxt("InvertedPendulum_results_AC.csv", delimiter=",")
all_results = [r]
labels = ["Actor Critic"]

# plot the results
%matplotlib inline
from hw3_utils import plot_results
fig = plot_results(all_results, labels)

# Question 4: Learning "Stabilizing the Ring" Strategies

In questions 2 and 3, we developed a few algorithms that can be used to solve continuous control problems. We now attempt to use these algorithms to solve a widely studied problem, and one that has been frequently referenced in class as well as previous homeworks: stabilizing traffic in a ring road.

### a. Designing an environment

We begin by designing an MDP that is representative of our problem. Using the Flow computational framework, this can be done by creating an environment object, similar to the one used to trained an optimal traffic light policy in the previous assignment.

In the below cell, design an environment than can be used to develop a controller that can dissipate the formation of stop-and-go waves in a ring road. In order to do so, perform the following tasks:

1. Modify the `observation_space` and `get_state` methods so that your state is the speed of the RL vehicle, as well the RL vehicle's headway and the speed of the vehicle ahead of it (i.e. its leader). The headway values in the state space should be clipped from above by `self.headway_normalizer` term. Moreover, the speeds and headways in the state space should be divided by the `self.speed_normalizer` and `self.headway_noramlizer` terms, respectively.
2. Modify the `action_space` and `_apply_rl_actions` methods so that your policy's actions are converted to desired accelerations by the RL vehicle in the environment. The actions as defined in the `action_space` method should *not* be bounded.

**Hint**:

- For a review of creating custom environments in flow, please see the following [tutorial](https://github.com/flow-project/flow/blob/master/tutorials/tutorial06_environments.ipynb).
- Individual vehicle state information can be collected from the Vehicles class within an environmnet (called by `self.vehicles`). Refer to [this file](https://github.com/flow-project/flow/blob/master/flow/core/vehicles.py) for what sort of information can be collected. The same could be done for scenario/network information using the variable `self.scenario`, which the associated get methods available [here](https://github.com/flow-project/flow/blob/master/flow/scenarios/base_scenario.py).

In [ ]:
from flow.envs import Env   # base environment class
from gym.spaces import Box  # used to define configuration of the state/action spaces
import numpy as np


class StabilizingTheRingEnv(Env):
    """Environment used to train ring stabilizing behavior.

    States
        The states are the speeds and headways of all vehicles.

    Actions
        The actions are an acceleration for each automated vehicle.

    Rewards
        The reward function rewards high average speeds from all vehicles in
        the network, and penalizes accelerations by the rl vehicle.

    Termination
        The rollout is terminate if the time horizon is met or if two 
        vehicles collide into one another.
    """
    def __init__(self, env_params, sumo_params, scenario):
        super().__init__(env_params, sumo_params, scenario)

        # constant to be divided from all speed values in the state space
        self.speed_normalizer = 30

        # constant to be divided from all headways in the state space
        self.headway_normalizer = 50
    
    @property
    def action_space(self):
        ##############################################################
        # specify dimensions and properties of the action space here #
        ##############################################################
        return  ### FILL IN ###

    @property
    def observation_space(self):
        #############################################################
        # specify dimensions and properties of the state space here #
        #############################################################
        return  ### FILL IN ###

    def get_state(self, **kwargs):
        ####################################
        # specify desired state space here #
        ####################################
        return  ### FILL IN ###

    def _apply_rl_actions(self, rl_actions):
        #####################################
        # specify desired action space here #
        #####################################
        ### FILL IN ###

    def compute_reward(self, state, rl_actions, **kwargs):
        # in the warmup steps
        if rl_actions is None:
            return 0

        vel = np.array([
            self.vehicles.get_speed(veh_id)
            for veh_id in self.vehicles.get_ids()
        ])

        if any(vel < -100) or kwargs["fail"]:
            return 0.

        # reward average velocity
        eta_2 = 4.
        reward = eta_2 * np.mean(vel) / 20

        # punish accelerations (should lead to reduced stop-and-go waves)
        eta = 8
        rl_actions = np.array(rl_actions)
        accel_threshold = 0

        if np.mean(np.abs(rl_actions)) > accel_threshold:
            reward += eta * (accel_threshold - np.mean(np.abs(rl_actions)))

        return float(reward)

### b. Training and testing

Once your environment is ready, you can train your environment by running the scripts below. Each cell is responsible for running a different algorithm, and will save the data and model from the most recent run, so feel free to play around with the hyperparameters of each algorithm independently. After you have trained everything, be sure to plot the training curves from all experiments by executing the last cell in the group.

**Note**: Your best policy should get a max return greater than 300.

In [ ]:
### creating the gym environment
from hw3_utils import get_params, HORIZON

sumo_params, env_params, scenario = get_params()

env = StabilizingTheRingEnv(env_params=env_params, 
                            sumo_params=sumo_params, 
                            scenario=scenario)
env._max_episode_steps = HORIZON

In [ ]:
### training on REINFORCE
import numpy as np
alg = REINFORCE(env, stochastic=True)

# feel free to modify the hyperparameters
cum_rewards = alg.train(learning_rate=0.1, gamma=0.99,
                        steps_per_iteration=3000, num_iterations=300)

alg.save_checkpoint("REINFORCE.ckpt")
np.savetxt("REINFORCE.csv", cum_rewards, delimiter=",")

In [ ]:
### training on REINFORCE with a baseline
import numpy as np
alg = REINFORCE_baseline(env, stochastic=True)

# feel free to modify the hyperparameters
cum_rewards = alg.train(learning_rate=0.1, gamma=0.99,
                        steps_per_iteration=3000, num_iterations=300)

alg.save_checkpoint("REINFORCE_baseline.ckpt")
np.savetxt("REINFORCE_baseline.csv", cum_rewards, delimiter=",")

In [ ]:
### training on Actor Critic
import numpy as np
alg = ActorCritic(env, stochastic=True)

# feel free to modify the hyperparameters
cum_rewards = alg.train(learning_rate=0.1, gamma=0.99, 
                        steps_per_iteration=3000, num_iterations=300,
                        num_value_estimates=10, num_train_steps_per_estimate=10)

alg.save_checkpoint("ActorCritic.ckpt")
np.savetxt("ActorCritic.csv", cum_rewards, delimiter=",")

In [ ]:
### plotting results
import numpy as np
from hw3_utils import plot_results

res_REINFORCE = np.array([np.genfromtxt("REINFORCE.csv")])
res_REINFORCE_baseline = np.array([np.genfromtxt("REINFORCE_baseline.csv")])
res_Actor_Critic = np.array([np.genfromtxt("ActorCritic.csv")])
all_results = [res_REINFORCE, res_REINFORCE_baseline, res_Actor_Critic]
labels = ["REINFORCE", "REINFORCE Baseline", "Actor Critic"]

plot_results(all_results, labels)

Finally, test that at least one of your algorithms has successfully managed to dissipate the formation of stop and go waves in the ring road network. In order to do so, specify the policy that performed the best from your implementations above and rerun it on the network in the cell below. If your training algorithm was successful, the average speed of vehicles should remain near the higher dotted line in the generated figure below.

In [ ]:
###########################################################
# Specify the algorithm you would like to visualize here. #
#                                                         #
# Choose one of:                                          #
# - "REINFORCE"                                           #
# - "REINFORCE_baseline"                                  #
# - "ActorCritic"                                         #
###########################################################
alg_name = "ActorCritic"

# ===========================================================================
# Do not modify below here
# ===========================================================================

import matplotlib.pyplot as plt
import numpy as np
from hw3_utils import get_params, HORIZON

assert alg_name in ["REINFORCE", "REINFORCE_baseline", "ActorCritic"]

# modify the reward function of the environment to output average speeds:
class TestEnvironment(StabilizingTheRingEnv):
    def compute_reward(self, state, rl_actions, **kwargs):
        return np.mean(self.vehicles.get_speed(self.vehicles.get_ids()))

# create the gym environment
sumo_params, env_params, scenario = get_params(render=True)
env = TestEnvironment(env_params=env_params, 
                      sumo_params=sumo_params, 
                      scenario=scenario)
env._max_episode_steps = HORIZON

# run the environment for one rollout with the trained policy
alg = DirectPolicyAlgorithm(env, stochastic=True)
alg.load_checkpoint(alg_name + ".ckpt")
samples = alg.rollout()

# plot the results from that rollout
avg_speeds = samples["reward"]

plt.figure(figsize=(12,8))
plt.title("Stabilizing Controller Performance", fontsize=25)
plt.xlabel("time step", fontsize=20)
plt.ylabel("average speed (m/s)", fontsize=20)
plt.plot(avg_speeds, c='k', label="average speed with one AV")
plt.plot([4.82] * len(avg_speeds), '--', c='b', label="non-perturbed equilibrium")
plt.plot([3.28] * len(avg_speeds), '--', c='r', label="perturbed equilibrium")
plt.legend(fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.show()

# Question 5: Effect of Hyperparameters (BONUS)

Explore the effect of one or more hyperparamters (e.g. depth or width of your neural network, learning rate, etc...) on the training performance of one of your algorithms on the above task. Plot the training results for each choice hyperparameter in the cell below.

In [ ]:
############################################
#### Perform hyperparameter search here ####
############################################